In [2]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import LSTM, Dense, Bidirectional
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot

In [10]:
df=pd.read_csv('/content/drive/MyDrive/train.csv')
df.head()

,id,title,author,text,label,Unnamed: 5,Unnamed: 6
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,NaN,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN,NaN


In [11]:
df1=df.drop('Unnamed: 5',axis=1)


In [12]:
df1.head()

,id,title,author,text,label,Unnamed: 6
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,NaN
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,NaN
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,NaN
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,NaN
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,NaN


In [13]:
df=df1.drop('Unnamed: 6', axis=1)
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [15]:
df.isnull().values.sum()

2555

In [16]:
df.shape

(20800, 5)

In [17]:
df=df.dropna()

In [18]:
df.shape

(18284, 5)

In [19]:
df['title'].isnull().values.any()

False

In [20]:
message=df['title']
message.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [22]:
type(message[0])

str

In [24]:
import nltk
from nltk.corpus import stopwords
import re


In [25]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [28]:
X=df.drop('label',axis=1)

In [29]:
messages=X.copy()

In [48]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()


In [33]:
messages['title'].head(10)

0     House Dem Aide: We Didn’t Even See Comey’s Let...
1     FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                     Why the Truth Might Get You Fired
3     15 Civilians Killed In Single US Airstrike Hav...
4     Iranian woman jailed for fictional unpublished...
5     Jackie Mason: Hollywood Would Love Trump if He...
7     Benoît Hamon Wins French Socialist Party’s Pre...
9     A Back-Channel Plan for Ukraine and Russia, Co...
10    Obama’s Organizing for Action Partners with So...
11    BBC Comedy Sketch "Real Housewives of ISIS" Ca...
Name: title, dtype: object

In [34]:
message.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [35]:
message.head(10)

0     House Dem Aide: We Didn’t Even See Comey’s Let...
1     FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                     Why the Truth Might Get You Fired
3     15 Civilians Killed In Single US Airstrike Hav...
4     Iranian woman jailed for fictional unpublished...
5     Jackie Mason: Hollywood Would Love Trump if He...
7     Benoît Hamon Wins French Socialist Party’s Pre...
9     A Back-Channel Plan for Ukraine and Russia, Co...
10    Obama’s Organizing for Action Partners with So...
11    BBC Comedy Sketch "Real Housewives of ISIS" Ca...
Name: title, dtype: object

In [41]:
new_message=message[:6]
new_message.head()

0    House Dem Aide: We Didn’t Even See Comey’s Let...
1    FLYNN: Hillary Clinton, Big Woman on Campus - ...
2                    Why the Truth Might Get You Fired
3    15 Civilians Killed In Single US Airstrike Hav...
4    Iranian woman jailed for fictional unpublished...
Name: title, dtype: object

In [39]:
another_message=message[5:]
another_message.head()

5     Jackie Mason: Hollywood Would Love Trump if He...
7     Benoît Hamon Wins French Socialist Party’s Pre...
9     A Back-Channel Plan for Ukraine and Russia, Co...
10    Obama’s Organizing for Action Partners with So...
11    BBC Comedy Sketch "Real Housewives of ISIS" Ca...
Name: title, dtype: object

In [42]:
type(another_message)

pandas.core.series.Series

In [43]:
for msg in message:
  print(msg)

Streaming output truncated to the last 5000 lines.
Mitch McConnell, Paul Ryan Meet with Donald Trump Promise ‘Bold Agenda’ - Breitbart
Susan Rice, Ex-Obama Adviser, Is Back in Political Cross Hairs Over Surveillance - The New York Times
The Fix Is In: NBC Affiliate Accidentally Posts Election Results A Week Early: Hillary Wins Presidency 42% to Trump’s 40%
Original unreleased transcript of Sonia Gandhi’s interview with Rajdeep Sardesai obtained
Indian Man Accused in Multimillion-Dollar Call-Center Swindle Is Held - The New York Times
7 Teenage Deaths, but No Answers for Aboriginal Canadians - The New York Times
Fashioning Cast-Iron Pans for Today’s Cooks - The New York Times
Summer Sweat: Hollywood Braces for Possible Worst Box Office in Decade
Trump Executive Orders Target Trade Abuses Amid NAFTA Intrigue - Breitbart
New York Times’s Moscow Bureau Was Targeted by Hackers - The New York Times
Barbra Streisand Says Sexism Cost Her Multiple Oscar Nominations
Comey Investigation a Red Her

In [55]:
corpus=[]
for msg in message:
  msg=re.sub('[^a-z,A-Z]',' ', msg)
  corpus.append(msg)

In [56]:
corpus

['House Dem Aide  We Didn t Even See Comey s Letter Until Jason Chaffetz Tweeted It',
 'FLYNN  Hillary Clinton, Big Woman on Campus   Breitbart',
 'Why the Truth Might Get You Fired',
 '   Civilians Killed In Single US Airstrike Have Been Identified',
 'Iranian woman jailed for fictional unpublished story about woman stoned to death for adultery',
 'Jackie Mason  Hollywood Would Love Trump if He Bombed North Korea over Lack of Trans Bathrooms  Exclusive Video    Breitbart',
 'Beno t Hamon Wins French Socialist Party s Presidential Nomination   The New York Times',
 'A Back Channel Plan for Ukraine and Russia, Courtesy of Trump Associates   The New York Times',
 'Obama s Organizing for Action Partners with Soros Linked  Indivisible  to Disrupt Trump s Agenda',
 'BBC Comedy Sketch  Real Housewives of ISIS  Causes Outrage',
 'Russian Researchers Discover Secret Nazi Military Base  Treasure Hunter  in the Arctic  Photos ',
 'US Officials See No Link Between Trump and Russia',
 'Re  Yes, Th

In [57]:
corpus1=[]
for words in corpus:
  words=words.lower()
  corpus1.append(words)

In [58]:
corpus1

['house dem aide  we didn t even see comey s letter until jason chaffetz tweeted it',
 'flynn  hillary clinton, big woman on campus   breitbart',
 'why the truth might get you fired',
 '   civilians killed in single us airstrike have been identified',
 'iranian woman jailed for fictional unpublished story about woman stoned to death for adultery',
 'jackie mason  hollywood would love trump if he bombed north korea over lack of trans bathrooms  exclusive video    breitbart',
 'beno t hamon wins french socialist party s presidential nomination   the new york times',
 'a back channel plan for ukraine and russia, courtesy of trump associates   the new york times',
 'obama s organizing for action partners with soros linked  indivisible  to disrupt trump s agenda',
 'bbc comedy sketch  real housewives of isis  causes outrage',
 'russian researchers discover secret nazi military base  treasure hunter  in the arctic  photos ',
 'us officials see no link between trump and russia',
 're  yes, th

In [59]:
corpus3=[]
for words in corpus1:
  words=words.split()
  words=[ps.stem(word) for word in words if not word in stopwords.words('english')]

  words=' '.join(words)
  corpus3.append(words)

In [60]:
corpus3

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton, big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia, courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'yes, paid govern troll social media, blogs, forum websit',
 'major leagu soccer, argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi clo

In [61]:
message.shape

(18284,)

In [62]:
voc_size=5000
one_hot_rep=[one_hot(words, voc_size) for words in corpus3]
one_hot_rep

[[114, 283, 4104, 3837, 1539, 461, 4353, 4429, 2129, 2607],
 [617, 3790, 2048, 895, 530, 980, 4249],
 [4841, 278, 4978, 1628],
 [3477, 357, 1082, 615, 2663, 253],
 [2499, 530, 3610, 4369, 2089, 2444, 530, 4026, 4291, 1862],
 [3738,
  4735,
  1631,
  188,
  4215,
  3187,
  1965,
  4721,
  2682,
  1915,
  4142,
  2248,
  1362,
  4147,
  4249],
 [1099, 2554, 4442, 2902, 1783, 1502, 3675, 3079, 370, 1406, 1972],
 [3360, 2534, 2389, 4341, 4116, 3330, 3187, 3649, 370, 1406, 1972],
 [2268, 1507, 3794, 1832, 4603, 954, 1577, 3379, 3187, 3084],
 [682, 2367, 3479, 1208, 3006, 3954, 1210, 1657],
 [3340, 480, 2219, 3497, 126, 1654, 459, 4705, 3219, 2486, 2123],
 [615, 1367, 1539, 954, 3187, 4116],
 [508, 2728, 915, 334, 1917, 3246, 2265, 3196, 1650],
 [2169, 304, 3113, 2612, 2396, 1589, 3390, 370, 1406, 1972],
 [3177, 628, 1236, 600, 3909, 370, 1406, 1972],
 [161, 4109, 2668, 4991, 3177, 3319, 281, 1847, 4900, 2531],
 [362, 4513, 3790],
 [2980, 3254, 3742, 2031, 3187, 4732, 4683, 4249],
 [222, 314

In [64]:
max_length=20
embedded_string=pad_sequences(one_hot_rep, padding='pre', maxlen=max_length)
embedded_string

array([[   0,    0,    0, ..., 4429, 2129, 2607],
       [   0,    0,    0, ...,  530,  980, 4249],
       [   0,    0,    0, ...,  278, 4978, 1628],
       ...,
       [   0,    0,    0, ...,  370, 1406, 1972],
       [   0,    0,    0, ..., 4469, 1169, 2119],
       [   0,    0,    0, ...,  297, 1122,  298]], dtype=int32)

In [70]:
from tensorflow.keras.layers import Embedding, Dropout

In [115]:
n_features=40
model=Sequential()
model.add(Embedding(voc_size, n_features, input_length=max_length))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_2 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [74]:
y=df['label']

In [75]:
len(embedded_string), y.shape

(18284, (18284,))

In [76]:
X_input=np.array(embedded_string)
y_input=np.array(y)

In [77]:
X_input.shape, y_input.shape

((18284, 20), (18284,))

In [80]:
type(y_input[1])

str

In [83]:
new_y=[]
for vals in y:
  if vals=='1':
    new_y.append(int(vals))
  elif vals=='0':
    new_y.append(int(vals))
  else:
    new_y.append(0)
new_y

[1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,


In [84]:
new_y_input=np.array(new_y)

In [85]:
type(new_y_input)

numpy.ndarray

In [86]:
type(new_y_input[0])

numpy.int64

In [116]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split( X_input, new_y_input, test_size=0.3, random_state=101)

In [117]:
model.fit(X_train, y_train,validation_data=(X_test, y_test), epochs=10, verbose=1, batch_size=64)

Epoch 1/10
200/200 [==============================] - 14s 52ms/step - loss: 0.3077 - accuracy: 0.8576 - val_loss: 0.2049 - val_accuracy: 0.9162
Epoch 2/10
200/200 [==============================] - 10s 48ms/step - loss: 0.1493 - accuracy: 0.9444 - val_loss: 0.1929 - val_accuracy: 0.9185
Epoch 3/10
200/200 [==============================] - 10s 49ms/step - loss: 0.1075 - accuracy: 0.9630 - val_loss: 0.2255 - val_accuracy: 0.9162
Epoch 4/10
200/200 [==============================] - 10s 49ms/step - loss: 0.0847 - accuracy: 0.9715 - val_loss: 0.2486 - val_accuracy: 0.9165
Epoch 5/10
200/200 [==============================] - 10s 48ms/step - loss: 0.0642 - accuracy: 0.9792 - val_loss: 0.3395 - val_accuracy: 0.9134
Epoch 6/10
200/200 [==============================] - 10s 49ms/step - loss: 0.0498 - accuracy: 0.9844 - val_loss: 0.3460 - val_accuracy: 0.9107
Epoch 7/10
200/200 [==============================] - 10s 49ms/step - loss: 0.0345 - accuracy: 0.9897 - val_loss: 0.3843 - val_accuracy:

In [103]:
from tensorflow.keras.models import Sequential

In [119]:
preed_y=model.predict(X_test)

In [120]:
print(preed_y)

[[2.4685264e-03]
 [7.0020842e-06]
 [2.0478228e-05]
 ...
 [6.9417358e-05]
 [9.9740458e-01]
 [2.7688325e-06]]


In [143]:
pred_y=(model.predict(X_test) > 0.5).astype("int32") #This can be used in place of model.predict_classes

In [150]:
demo_pred=(model.predict(X_test)>0.5)
demo_pred

array([[False],
       [False],
       [False],
       ...,
       [False],
       [ True],
       [False]])

In [144]:
pred_y

array([[0],
       [0],
       [0],
       ...,
       [0],
       [1],
       [0]], dtype=int32)

In [132]:
pred_y.shape

(5486, 1)

In [139]:
X_test.shape

(5486, 20)

In [145]:
pred_y=pred_y.flatten()

In [146]:
pred_y

array([0, 0, 0, ..., 0, 1, 0], dtype=int32)

In [147]:
pred_y.shape

(5486,)

In [133]:
type(pred_y)

numpy.ndarray

In [107]:
#predicting the value based on the model learning
predict_x=model.predict(X_test) 
classes_x=np.argmax(predict_x,axis=1)

In [114]:
(unique, counts) = np.unique(classes_x, return_counts=True)
frequencies = np.asarray((unique, counts)).T


print(frequencies)

[[   0 5486]]


In [91]:
y_test

array([0, 0, 0, ..., 0, 1, 0])

In [148]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test, pred_y))

[[2853  284]
 [ 212 2137]]


In [149]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,pred_y)

0.9095880422894641